December 10, 2024

Goal: Characterize the benchmark circuits in `feynman/benchmarks/qasm`

In [1]:
cd ..

/home/abhishekabhishek/git/cpsc-513-project


In [2]:
benchmark_dir_path = "../feynman/benchmarks/qasm/"

import glob
file_path_list = glob.glob(benchmark_dir_path + "*.qasm")
file_path_list

['../feynman/benchmarks/qasm/csla_mux_3.qasm',
 '../feynman/benchmarks/qasm/hwb10.qasm',
 '../feynman/benchmarks/qasm/barenco_tof_10.qasm',
 '../feynman/benchmarks/qasm/gf2^32_mult.qasm',
 '../feynman/benchmarks/qasm/gf2^10_mult.qasm',
 '../feynman/benchmarks/qasm/gf2^5_mult.qasm',
 '../feynman/benchmarks/qasm/grover_5.qasm',
 '../feynman/benchmarks/qasm/mod_mult_55.qasm',
 '../feynman/benchmarks/qasm/gf2^6_mult.qasm',
 '../feynman/benchmarks/qasm/tof_4.qasm',
 '../feynman/benchmarks/qasm/barenco_tof_4.qasm',
 '../feynman/benchmarks/qasm/ham15-med.qasm',
 '../feynman/benchmarks/qasm/ham15-high.qasm',
 '../feynman/benchmarks/qasm/tof_3.qasm',
 '../feynman/benchmarks/qasm/qcla_com_7.qasm',
 '../feynman/benchmarks/qasm/tof_10.qasm',
 '../feynman/benchmarks/qasm/hwb11.qasm',
 '../feynman/benchmarks/qasm/mod_red_21.qasm',
 '../feynman/benchmarks/qasm/tof_5.qasm',
 '../feynman/benchmarks/qasm/barenco_tof_5.qasm',
 '../feynman/benchmarks/qasm/gf2^8_mult.qasm',
 '../feynman/benchmarks/qasm/qcl

In [5]:
from keys import IBMQ_API
from qiskit import QuantumCircuit, transpile
from qiskit.qasm2 import QASM2ParseError
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum", token=IBMQ_API, instance="ibm-q/open/main")
ibm_backend = service.backend("ibm_sherbrooke")
backend_qubits = ibm_backend.num_qubits

In [6]:
circ_info_dict = {
    'name': [],
    'n_gates_original': [],
    'n_gates_transpiled': [],
    'depth_original': [],
    'depth_transpiled': [],
    'gate_set': [],
    'n_qubits_original': [],
    'n_qubits_transpiled': [],
    'n_ancilla_original': [],
    'n_ancilla_transpiled': []
}

for file_path in file_path_list:
    # build the circuit from the .qasm file
    try:
        circ = QuantumCircuit.from_qasm_file(file_path)
        
        # check the number of qubits in the original circuit, ignore if circuit is larger than what
        # can be fit on the backend
        circ_total_qubits = circ.num_qubits + circ.num_ancillas
        if circ_total_qubits > backend_qubits:
            pass
        else:
            # extract the name of the benchmark
            circ_info_dict['name'].append(
                file_path.replace(benchmark_dir_path, '', 1).replace('.qasm', '', 1)
            )

            # get the metrics from the original "uncompiled" circuit
            circ_op_dict = circ.count_ops()
            circ_info_dict['gate_set'].append('_'.join(list(circ_op_dict.keys())))
            
            circ_info_dict['n_qubits_original'].append(circ.num_qubits)
            circ_info_dict['n_gates_original'].append(sum(circ_op_dict.values()))
            circ_info_dict['depth_original'].append(circ.depth())
            circ_info_dict['n_ancilla_original'].append(circ.num_ancillas)
            
            # transpile the circuit to the IBM backend (ibm_sherbrooke) and get the metrics from the
            # transpiled circuit
            circ_hw = transpile(circ, ibm_backend)
            
            circ_hw_op_dict = circ_hw.count_ops()
            circ_info_dict['n_qubits_transpiled'].append(circ_hw.num_qubits)
            circ_info_dict['n_gates_transpiled'].append(sum(circ_hw_op_dict.values()))
            circ_info_dict['depth_transpiled'].append(circ_hw.depth())
            circ_info_dict['n_ancilla_transpiled'].append(circ_hw.num_ancillas)
    
    except QASM2ParseError:
        print(f"Unable to parse {file_path} due to QASM2ParseError.")

Unable to parse ../feynman/benchmarks/qasm/mod_adder_1048576.qasm due to QASM2ParseError.
Unable to parse ../feynman/benchmarks/qasm/cycle_17_3.qasm due to QASM2ParseError.


In [7]:
import pandas as pd
circ_info_df = pd.DataFrame.from_dict(circ_info_dict)
circ_info_df

,name,n_gates_original,n_gates_transpiled,depth_original,depth_transpiled,gate_set,n_qubits_original,n_qubits_transpiled,n_ancilla_original,n_ancilla_transpiled
0,csla_mux_3,70,1052,24,428,h_cx_ccx,15,127,0,0
1,hwb10,31764,690262,18050,270667,h_cx_ccx_x,16,127,0,0
2,barenco_tof_10,130,2948,98,1603,h_ccx,19,127,0,0
3,gf2^32_mult,3322,120314,261,20042,h_ccx_cx,96,127,0,0
4,gf2^10_mult,347,11224,77,2543,h_ccx_cx,30,127,0,0
5,gf2^5_mult,97,2424,38,901,h_ccx_cx,15,127,0,0
6,grover_5,351,3828,160,2021,h_x_ccx,9,127,0,0
7,mod_mult_55,49,610,20,246,h_x_ccx_cx,9,127,0,0
8,gf2^6_mult,135,3661,47,1254,h_ccx_cx,18,127,0,0
9,tof_4,25,334,17,191,h_ccx,7,127,0,0


In [ ]:
# drop the n_ancilla columns from the df
circ_info_df = circ_info_df.drop(columns=['n_ancilla_original', 'n_ancilla_transpiled'])
circ_info_df

,name,n_gates_original,n_gates_transpiled,depth_original,depth_transpiled,gate_set,n_qubits_original,n_qubits_transpiled
0,csla_mux_3,70,1052,24,428,h_cx_ccx,15,127
1,hwb10,31764,690262,18050,270667,h_cx_ccx_x,16,127
2,barenco_tof_10,130,2948,98,1603,h_ccx,19,127
3,gf2^32_mult,3322,120314,261,20042,h_ccx_cx,96,127
4,gf2^10_mult,347,11224,77,2543,h_ccx_cx,30,127
5,gf2^5_mult,97,2424,38,901,h_ccx_cx,15,127
6,grover_5,351,3828,160,2021,h_x_ccx,9,127
7,mod_mult_55,49,610,20,246,h_x_ccx_cx,9,127
8,gf2^6_mult,135,3661,47,1254,h_ccx_cx,18,127
9,tof_4,25,334,17,191,h_ccx,7,127


In [9]:
# sort the df by the number of qubits in the original circuit
circ_info_df = circ_info_df.sort_values(by='n_qubits_original', ascending=True)
circ_info_df

,name,n_gates_original,n_gates_transpiled,depth_original,depth_transpiled,gate_set,n_qubits_original,n_qubits_transpiled
13,tof_3,15,182,11,116,h_ccx,5,127
29,barenco_tof_3,20,282,14,159,h_ccx,5,127
25,qft_4,159,595,134,365,h_t_cx_s_tdg_sdg_ccx,5,127
36,mod5_4,23,283,23,179,h_ccx_cx_x,5,127
10,barenco_tof_4,34,577,26,308,h_ccx,7,127
31,hwb6,109,1528,62,833,h_cx_ccx_x,7,127
9,tof_4,25,334,17,191,h_ccx,7,127
7,mod_mult_55,49,610,20,246,h_x_ccx_cx,9,127
19,barenco_tof_5,50,969,38,532,h_ccx,9,127
18,tof_5,35,494,23,276,h_ccx,9,127


In [10]:
# save the df to disk to save repeated work computing the above properties
circ_info_df.to_csv('feynman_benchmark_properties.csv')